# Modules

In [19]:
import tensorflow as tf
import scipy
from tensorflow.python.client import device_lib
from tensorflow.keras.applications import InceptionV3, ResNet152V2, VGG19
from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D, BatchNormalization,AveragePooling2D, TimeDistributed, LSTM
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow as tf
print(tf.__version__)

2.10.1


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4927548939509417643
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2240439911
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4895957197095910743
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:07:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

# Dataset preparation

In [5]:
batchsize = 8

In [6]:
# Training dataset

train_data_gen = ImageDataGenerator(rotation_range=0.2, 
                                        shear_range=0.2,
                                        zoom_range=0.2, 
                                        width_shift_range=0.2,
                                        height_shift_range=0.2, 
                                        rescale=1./255,
                                        validation_split=0.2)

train_data = train_data_gen.flow_from_directory('dataset/train', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical',
                                                    subset='training'
                                                    )

validation_data = train_data_gen.flow_from_directory('dataset/train', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical',
                                                    subset='validation'
                                                    )

Found 64719 images belonging to 2 classes.
Found 16179 images belonging to 2 classes.


In [8]:
# Testing dataset

test_data_gen = ImageDataGenerator(rescale=1./255)
test_data = test_data_gen.flow_from_directory('dataset/test', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical'
                                                    )

Found 4000 images belonging to 2 classes.


# Model preparation

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [69]:
# Base Model : InceptionV3

# Get knowledge from InceptionV3 pretrained model, so we don't need to train the model from scratch
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)

# The only layers we need to train
# This is the layers' illustration https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Fthe-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480&psig=AOvVaw0CYHQ2oruhWMmKK2Lxm5cN&ust=1668996857968000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCLDJs9TYu_sCFQAAAAAdAAAAABAU
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)

for layer in bmodel.layers:
    layer.trainable = False

checkpoint = ModelCheckpoint('models/InceptionV3', 
                             monitor='val_loss', 
                             save_best_only=True, 
                             verbose=3)
    
print('done')

done


In [241]:
# Base Model : ResNet152V2

# # Get knowledge from ResNet152V2 pretrained model, so we don't need to train the model from scratch
# bmodel = ResNet152V2(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
# hmodel = bmodel.output
# hmodel = AveragePooling2D(pool_size = (3,3))(hmodel)
# hmodel = Flatten(name= 'flatten')(hmodel)

# # The only layers we need to train
# # This is the layers' illustration https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Fthe-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480&psig=AOvVaw0CYHQ2oruhWMmKK2Lxm5cN&ust=1668996857968000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCLDJs9TYu_sCFQAAAAAdAAAAABAU

# hmodel = Dense(256, activation = "relu")(hmodel)
# hmodel = Dropout(0.25)(hmodel)
# hmodel = Dense(256, activation = "relu")(hmodel)
# hmodel = Dropout(0.25)(hmodel)
# hmodel = Dense(256, activation = "relu")(hmodel)
# hmodel = Dropout(0.25)(hmodel)
# hmodel = Dense(2, activation='softmax')(hmodel)

# model = Model(inputs=bmodel.input, outputs=hmodel)


# for layer in bmodel.layers:
#     layer.trainable = False

# checkpoint = ModelCheckpoint('models/ResNet152V2', 
#                              monitor='val_loss', 
#                              save_best_only=True, 
#                              verbose=3)

# print('done')

done


In [48]:
# Base Model : CNN LSTM
# Did some modifications according to https://www.tensorflow.org/versions/r2.2/api_docs/python/tf/keras/layers/LSTM

bmodelVGG = VGG19(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel1 = Sequential()
hmodel1 = bmodelVGG.output
hmodel1 = AveragePooling2D(pool_size =(3,3), padding='same')(hmodel1)
hmodel1 = TimeDistributed(Flatten())(hmodel1)

#Train this layer 
hmodel1 = LSTM(64, dropout=0.5,  recurrent_dropout=0, unroll=False, activation='tanh')(hmodel1)
hmodel1 = BatchNormalization()(hmodel1)
hmodel1 = Dropout(0.25)(hmodel1)
hmodel1 = Dense(32, activation = "relu", input_shape=())(hmodel1)
hmodel1 = Dropout(0.25)(hmodel1)
hmodel1 = Dense(16, activation = "relu")(hmodel1)
hmodel1 = Dropout(0.25)(hmodel1)
hmodel1 = Dense(8, activation = "relu")(hmodel1)
hmodel1 = Dropout(0.25)(hmodel1)
hmodel1 = Dense(1, activation='sigmoid')(hmodel1)
model = Model(inputs=bmodelVGG.input, outputs=hmodel1)

for layer in bmodelVGG.layers:
    layer.trainable = False

checkpoint = ModelCheckpoint('models/CNN_LSTM', 
                             monitor='val_loss', 
                             save_best_only=True, 
                             verbose=3)

print('done')

done


In [62]:
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_282 (Conv2D)            (None, 39, 39, 32)   864         ['input_23[0][0]']               
                                                                                                  
 batch_normalization_294 (Batch  (None, 39, 39, 32)  96          ['conv2d_282[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_282 (Activation)    (None, 39, 39, 32)   0           ['batch_normalization_294[

                                                                                                  
 batch_normalization_304 (Batch  (None, 7, 7, 96)    288         ['conv2d_292[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_305 (Batch  (None, 7, 7, 32)    96          ['conv2d_293[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_287 (Activation)    (None, 7, 7, 64)     0           ['batch_normalization_299[0][0]']
                                                                                                  
 activation_289 (Activation)    (None, 7, 7, 64)     0           ['batch_normalization_301[0][0]']
          

                                                                                                  
 conv2d_302 (Conv2D)            (None, 7, 7, 48)     13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_305 (Conv2D)            (None, 7, 7, 96)     55296       ['activation_304[0][0]']         
                                                                                                  
 batch_normalization_314 (Batch  (None, 7, 7, 48)    144         ['conv2d_302[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_317 (Batch  (None, 7, 7, 96)    288         ['conv2d_305[0][0]']             
 Normalization)                                                                                   
          

                                                                  'activation_311[0][0]',         
                                                                  'max_pooling2d_14[0][0]']       
                                                                                                  
 conv2d_316 (Conv2D)            (None, 3, 3, 128)    98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_328 (Batch  (None, 3, 3, 128)   384         ['conv2d_316[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_316 (Activation)    (None, 3, 3, 128)    0           ['batch_normalization_328[0][0]']
                                                                                                  
 conv2d_31

 batch_normalization_338 (Batch  (None, 3, 3, 160)   480         ['conv2d_326[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_326 (Activation)    (None, 3, 3, 160)    0           ['batch_normalization_338[0][0]']
                                                                                                  
 conv2d_327 (Conv2D)            (None, 3, 3, 160)    179200      ['activation_326[0][0]']         
                                                                                                  
 batch_normalization_339 (Batch  (None, 3, 3, 160)   480         ['conv2d_327[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activatio

 conv2d_337 (Conv2D)            (None, 3, 3, 160)    179200      ['activation_336[0][0]']         
                                                                                                  
 batch_normalization_349 (Batch  (None, 3, 3, 160)   480         ['conv2d_337[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_337 (Activation)    (None, 3, 3, 160)    0           ['batch_normalization_349[0][0]']
                                                                                                  
 conv2d_333 (Conv2D)            (None, 3, 3, 160)    122880      ['mixed5[0][0]']                 
                                                                                                  
 conv2d_338 (Conv2D)            (None, 3, 3, 160)    179200      ['activation_337[0][0]']         
          

 activation_347 (Activation)    (None, 3, 3, 192)    0           ['batch_normalization_359[0][0]']
                                                                                                  
 conv2d_343 (Conv2D)            (None, 3, 3, 192)    147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_348 (Conv2D)            (None, 3, 3, 192)    258048      ['activation_347[0][0]']         
                                                                                                  
 batch_normalization_355 (Batch  (None, 3, 3, 192)   576         ['conv2d_343[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_360 (Batch  (None, 3, 3, 192)   576         ['conv2d_348[0][0]']             
 Normaliza

                                                                                                  
 batch_normalization_364 (Batch  (None, 3, 3, 192)   576         ['conv2d_352[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_368 (Batch  (None, 3, 3, 192)   576         ['conv2d_356[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_352 (Activation)    (None, 3, 3, 192)    0           ['batch_normalization_364[0][0]']
                                                                                                  
 activation_356 (Activation)    (None, 3, 3, 192)    0           ['batch_normalization_368[0][0]']
          

 activation_360 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_372[0][0]']
                                                                                                  
 activation_361 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_373[0][0]']
                                                                                                  
 activation_364 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_376[0][0]']
                                                                                                  
 activation_365 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_377[0][0]']
                                                                                                  
 batch_normalization_378 (Batch  (None, 1, 1, 192)   576         ['conv2d_366[0][0]']             
 Normalization)                                                                                   
          

 activation_374 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_386[0][0]']
                                                                                                  
 batch_normalization_387 (Batch  (None, 1, 1, 192)   576         ['conv2d_375[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_367 (Activation)    (None, 1, 1, 320)    0           ['batch_normalization_379[0][0]']
                                                                                                  
 mixed9_1 (Concatenate)         (None, 1, 1, 768)    0           ['activation_369[0][0]',         
                                                                  'activation_370[0][0]']         
                                                                                                  
 concatena

# Train the model

In [70]:
# patience = max num of epochs when accuracy is not increasing
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

callbacks = [checkpoint, earlystop, learning_rate]

In [ ]:
# Resnet152V2

# try:
#     model.compile(optimizer='', loss='categorical_crossentropy', metrics=['accuracy'])

#     model.fit(train_data,
#           steps_per_epoch=train_data.samples//batchsize,
#           validation_data=validation_data,
#           validation_steps=validation_data.samples//batchsize,
#           callbacks=callbacks,
#           epochs=50)
    
#     model.save('')
# except Exception as e:
#     print(e)

In [71]:
# InceptionV3

try:
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data,
          steps_per_epoch=train_data.samples//batchsize,
          validation_data=validation_data,
          validation_steps=validation_data.samples//batchsize,
          callbacks=callbacks,
          epochs=50)
    
    model.save('InceptionV3_Drowsiness.h5')
except Exception as e:
    print(e)

Epoch 1/50
8089/8089 [==============================] - ETA: 0s - loss: 0.3256 - accuracy: 0.8822
Epoch 1: val_loss improved from inf to 0.49945, saving model to models\InceptionV3


INFO:tensorflow:Assets written to: models\InceptionV3\assets


INFO:tensorflow:Assets written to: models\InceptionV3\assets


8089/8089 [==============================] - 443s 54ms/step - loss: 0.3256 - accuracy: 0.8822 - val_loss: 0.4995 - val_accuracy: 0.8157 - lr: 0.0010
Epoch 2/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.2848 - accuracy: 0.9072
Epoch 2: val_loss did not improve from 0.49945
8089/8089 [==============================] - 498s 62ms/step - loss: 0.2848 - accuracy: 0.9072 - val_loss: 0.5343 - val_accuracy: 0.8030 - lr: 0.0010
Epoch 3/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.2743 - accuracy: 0.9097
Epoch 3: val_loss improved from 0.49945 to 0.36467, saving model to models\InceptionV3


INFO:tensorflow:Assets written to: models\InceptionV3\assets


...

INFO:tensorflow:Assets written to: models\InceptionV3\assets


8089/8089 [==============================] - 629s 78ms/step - loss: 0.2645 - accuracy: 0.9275 - val_loss: 0.3120 - val_accuracy: 0.9162 - lr: 1.0000e-04
Epoch 18/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2662 - accuracy: 0.9274
Epoch 18: val_loss did not improve from 0.31196
8089/8089 [==============================] - 559s 69ms/step - loss: 0.2662 - accuracy: 0.9274 - val_loss: 0.3212 - val_accuracy: 0.9104 - lr: 1.0000e-04
Epoch 19/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2677 - accuracy: 0.9286
Epoch 19: val_loss did not improve from 0.31196
8089/8089 [==============================] - 565s 70ms/step - loss: 0.2677 - accuracy: 0.9286 - val_loss: 0.3142 - val_accuracy: 0.9065 - lr: 1.0000e-04
Epoch 20/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2528 - accuracy: 0.9279
Epoch 20: val_loss improved from 0.31196 to 0.29997, saving model to models\InceptionV3


INFO:tensorflow:Assets written to: models\InceptionV3\assets


INFO:tensorflow:Assets written to: models\InceptionV3\assets


8089/8089 [==============================] - 605s 75ms/step - loss: 0.2528 - accuracy: 0.9279 - val_loss: 0.3000 - val_accuracy: 0.9165 - lr: 1.0000e-04
Epoch 21/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2681 - accuracy: 0.9300
Epoch 21: val_loss did not improve from 0.29997
8089/8089 [==============================] - 576s 71ms/step - loss: 0.2681 - accuracy: 0.9300 - val_loss: 0.3210 - val_accuracy: 0.9094 - lr: 1.0000e-04
Epoch 22/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2839 - accuracy: 0.9290
Epoch 22: val_loss did not improve from 0.29997
8089/8089 [==============================] - 558s 69ms/step - loss: 0.2839 - accuracy: 0.9290 - val_loss: 0.3109 - val_accuracy: 0.9167 - lr: 1.0000e-04
Epoch 23/50
8089/8089 [==============================] - ETA: 0s - loss: 0.3270 - accuracy: 0.9303
Epoch 23: val_loss did not improve from 0.29997

Epoch 23: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
8089/8089 [==========

In [47]:
# # CNN-LSTM

# try:
#     model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

#     model.fit(train_data,
#           steps_per_epoch=train_data.samples//batchsize,
#           validation_data=validation_data,
#           validation_steps=validation_data.samples//batchsize,
#           callbacks=callbacks,
#           epochs=50)
    
#     model.save('CNN_LSTM_Drowsiness.h5')
# except Exception as e:
#     print(e)

Epoch 1/50
Graph execution error:

Detected at node 'gradient_tape/model_10/dense_43/MatMul/MatMul' defined at (most recent call last):
    File "C:\Users\felix\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\felix\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\felix\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\felix\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\felix\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\felix\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\felix\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._r

# Model evaluation

In [72]:
acc_training, loss_training = model.evaluate(train_data)
print(acc_training)
print(loss_training)

8090/8090 [==============================] - 412s 51ms/step - loss: 0.2751 - accuracy: 0.9390
0.2751415967941284
0.9389514923095703


In [73]:
acc_val, loss_val = model.evaluate(validation_data)
print(acc_val)
print(loss_val)

2023/2023 [==============================] - 104s 51ms/step - loss: 0.3058 - accuracy: 0.9130
0.30575045943260193
0.9130353927612305


In [74]:
from tensorflow.keras.models import load_model
model = load_model('models/InceptionV3//')
acc_test, loss_test = model.evaluate(test_data)
print(acc_test)
print(loss_test)

500/500 [==============================] - 29s 55ms/step - loss: 0.5321 - accuracy: 0.8390
0.5320631265640259
0.8389999866485596
